<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/%ED%98%9C%EC%A0%95/torch_load_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공용폴더에 폴더(yolov5), 파일(model.pt) 업로드해뒀습니다.

In [ ]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/team2/BA_all_.pkl')
# df.columns
ba_mean = df.BONE.mean()
ba_std = df.BONE.std()

In [ ]:
# 필요 경로 및 gender 변수
img_path ='/content/drive/MyDrive/team2/preprocessing_done/female/312_F.jpg'
model_path = '/content/drive/MyDrive/team2/yolov5'
tjnet_path = '/content/drive/MyDrive/team2/TJM/tjnet4.h5'
gender = 0

# grobal : crops, X(x_test), img, pred
predict_zscore(gender, img_path, model_path, tjnet_path, show_crop=False)

In [ ]:
# 312_F
cond1 = df['gender'] == 0
cond2 = df['NO']==312
print('df z_score :', df[cond1&cond2]['BA_zscore'].values[0], ', 예측 : ', pred)

In [ ]:
# 예측된 z_score(pred)로 뼈나이 변환
# pred global 변수
(pred * ba_std) + ba_mean

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# img global 변수로 설정.
plt.figure(figsize=(16,10))
plt.imshow(img)
plt.show()

## 사용자함수

In [ ]:
def predict_zscore(gender, img_path, model_path,tjnet_path, show_crop=True):
  yolo_crop_img(gender, img_path, model_path, show_crop=show_crop)
  import numpy as np
  import tensorflow.keras as tf
  model = tf.models.load_model(tjnet_path, compile=False)
  # grobal X : X_ray 이미지의 yolo crop image 값
  y_predict = model.predict(X)
  global pred
  pred = y_predict[0][0]
  print('예측 z_score: ', pred)


def yolo_crop_img(gender, img_path, model_path, result_out=False, show_crop=False):
  import os
  import numpy as np
  os.chdir(model_path)
  import torch
  model = torch.load('./model.pt', map_location='cpu')
  result = model(img_path)
  global crops
  crops = result.crop(save=False)
  global X
  X = out_crop_img(crops, gender)
  global img
  img = np.squeeze(result.render())
  if show_crop: show_img(result)
  if result_out: return result

def show_img(result):
  import matplotlib.pyplot as plt
  import numpy as np
  %matplotlib inline
  plt.figure(figsize=(16,12))
  plt.imshow(np.squeeze(result.render()))
  plt.show()

def out_crop_img(crop, gender):
  import re
  import cv2
  import numpy as np

  gender = np.array(gender).reshape(1,1)

  for i in range(7):
    carpal = re.compile('CARPAL.')
    ip = re.compile('IP.')
    lmcp = re.compile('LMCP.')
    lpip = re.compile('LPIP.')
    mmcp = re.compile('MMCP.')
    mpip = re.compile('MPIP.')
    tmcp = re.compile('TMCP.')

    if carpal.search(crop[i]['label']):
      CARPAL_img = crop[i]['im']
      CARPAL_img = cv2.resize(CARPAL_img, (224,224),cv2.INTER_AREA)
      CARPAL_img = np.expand_dims(CARPAL_img, axis=0)

    if ip.search(crop[i]['label']):
      IP_img = crop[i]['im']
      IP_img = cv2.resize(IP_img, (75,75),cv2.INTER_AREA)
      IP_img = np.expand_dims(IP_img, axis=0)
      
    if lmcp.search(crop[i]['label']):
      LMCP_img = crop[i]['im']
      LMCP_img = cv2.resize(LMCP_img, (75,75),cv2.INTER_AREA)
      LMCP_img = np.expand_dims(LMCP_img, axis=0)

    if lpip.search(crop[i]['label']):
      LPIP_img = crop[i]['im']
      LPIP_img = cv2.resize(LPIP_img, (75,75),cv2.INTER_AREA)
      LPIP_img = np.expand_dims(LPIP_img, axis=0)
    
    if mmcp.search(crop[i]['label']):
      MMCP_img = crop[i]['im']
      MMCP_img = cv2.resize(MMCP_img, (75,75),cv2.INTER_AREA)
      MMCP_img = np.expand_dims(MMCP_img, axis=0)
    
    if mpip.search(crop[i]['label']):
      MPIP_img = crop[i]['im']
      MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)
      MPIP_img = np.expand_dims(MPIP_img, axis=0)

    if tmcp.search(crop[i]['label']):
      TMCP_img = crop[i]['im']
      TMCP_img = cv2.resize(TMCP_img, (75,75),cv2.INTER_AREA)
      TMCP_img = np.expand_dims(TMCP_img, axis=0)

    else : continue

  
  return [CARPAL_img, LMCP_img, MMCP_img,TMCP_img, LPIP_img, MPIP_img, IP_img, gender]

## -

In [ ]:
for i in range(7):
  print(crops[i]['label'])

In [ ]:
# 배열에 차원 추가, 3가지 방법
# CNN에서 이미지 파일을 불러와서 다차원 배열로 변환할 때 사용하곤 합니다.
# 출처 : https://rfriend.tistory.com/428

i = 5
import numpy as np
import re
import cv2
mpip = re.compile('MPIP.')
mpip.search(crops[i]['label'])
MPIP_img = crops[i]['im']
MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)

# 1)
MPIP_img1 = MPIP_img.reshape(1, 75, 75, 3)
print(MPIP_img1.shape)

# 2)
MPIP_img2 = np.expand_dims(MPIP_img, axis=0)
print(MPIP_img2.shape)

# 3)
MPIP_img3 = MPIP_img[np.newaxis,:]
print(MPIP_img3.shape)

In [ ]:
# # 필요한 패키지
# import os
# # 현재 경로 확인
# os.getcwd()
# # 경로 설정
# os.chdir('/content/drive/MyDrive/team2/yolov5')
# %cp -r /content/drive/MyDrive/model.pt /content/drive/MyDrive/team2/yolov5/model.pt

In [ ]:
import numpy as np
import tensorflow.keras as tf

model = tf.models.load_model(tjnet_path, compile=False)

In [ ]:
# grobal X : X_ray 이미지의 tjnet input 값
y_predict = model.predict(X)
print('예측 z_score: ', y_predict[0][0])

In [ ]:
# label = labels[y_predict[0].argmax()]
# confidence = y_predict[0][y_predict[0].argmax()]